# Feature Engineering

1.	Create Aggregate Features
	Example:
-	Total Transaction Amount: Sum of all transaction amounts for each customer.
-	Average Transaction Amount: Average transaction amount per customer.
-	Transaction Count: Number of transactions per customer.
-	Standard Deviation of Transaction Amounts: Variability of transaction amounts per customer.
2.	Extract Features
	Example:
-	Transaction Hour: The hour of the day when the transaction occurred.
-	Transaction Day: The day of the month when the transaction occurred.
-	Transaction Month: The month when the transaction occurred.
-	Transaction Year: The year when the transaction occurred.
3.	Encode Categorical Variables
Convert categorical variables into numerical format by using:
-	One-Hot Encoding: Converts categorical values into binary vectors.
-	Label Encoding: Assigns a unique integer to each category.
4.	Handle Missing Values
	Use imputation or Removal to handle missing values
-	Imputation: Filling missing values with mean, median, mode, or using more methods like KNN imputation.
-	Removal: Removing rows or columns with missing values if they are few.
5.	Normalize/Standardize Numerical Features
Normalization and standardization are scaling techniques used to bring all numerical features onto a similar scale.
-	Normalization: Scales the data to a range of [0, 1].
-	Standardization: Scales the data to have a mean of 0 and a standard deviation of 1.


In [53]:
import pandas as pd
import pytz
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from eda import *
import prediction as dp
from feature import *

## Load data

In [54]:
df , _ = load_data()

2024-10-08 22:42:57,929 - INFO - Loading Data ...
2024-10-08 22:42:58,479 - INFO - Loading Data Finshed


In [55]:
df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

## Aggregate 

In [56]:
df_agg = aggeregatef(df)

In [57]:
df_agg

,Transaction_count,Total_Transaction,Average_Transaction,Transaction_std
CustomerId,,,,
CustomerId_1,1,-10000.0,-10000.000000,NaN
CustomerId_10,1,-10000.0,-10000.000000,NaN
CustomerId_1001,5,20000.0,4000.000000,6558.963333
CustomerId_1002,11,4225.0,384.090909,560.498966
CustomerId_1003,6,20000.0,3333.333333,6030.478146
...,...,...,...,...
CustomerId_992,6,20000.0,3333.333333,6088.240030
CustomerId_993,5,20000.0,4000.000000,6745.368782
CustomerId_994,101,543873.0,5384.881188,14800.656784


In [132]:
df_agg.sort_values(by='Transaction_count')

,Transaction_count,Total_Transaction,Average_Transaction,Transaction_std
CustomerId,,,,
CustomerId_1,1,-10000.0,-10000.000000,NaN
CustomerId_2690,1,1000.0,1000.000000,NaN
CustomerId_2694,1,1000.0,1000.000000,NaN
CustomerId_2698,1,500.0,500.000000,NaN
CustomerId_2703,1,500000.0,500000.000000,NaN
...,...,...,...,...
CustomerId_4033,778,1768355.5,2272.950514,10382.687289
CustomerId_1096,784,1949226.0,2486.257653,17819.372757
CustomerId_647,1869,3633564.0,1944.121990,7715.389537


## Feature Extraction 

In [62]:
dff = df.copy()
# Convert the 'transaction_start_time' to datetime and localize to UTC
dff['TransactionStartTime'] = pd.to_datetime(dff['TransactionStartTime']).dt.tz_convert('UTC')

# # Convert to Uganda time (UTC+3)
# uganda_tz = pytz.timezone('Africa/Kampala')
# dff['TransactionStartTime_Uganda'] = dff['TransactionStartTime'].dt.tz_convert(uganda_tz)


In [63]:
dff['TransactionYear'] =  dff['TransactionStartTime'].dt.year
dff['TransactionMonth'] = dff['TransactionStartTime'].dt.month
dff['TransactionDay'] = dff['TransactionStartTime'].dt.day
dff['TransactionHour'] = dff['TransactionStartTime'].dt.hour

In [64]:
# Find the start and end dates
start_date = dff['TransactionStartTime'].min()
end_date = dff['TransactionStartTime'].max()

# Display the results
print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2018-11-15 02:18:49+00:00
End Date: 2019-02-13 10:01:28+00:00


In [68]:
dff['rec'] = end_date - dff['TransactionStartTime'] 
dff['Recency'] = dff['rec'].dt.days

In [115]:

# Calculate Recency
dff['recency'] = (end_date - dff.groupby('CustomerId')['TransactionStartTime_Uganda'].max()).dt.days

In [65]:
dff.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'TransactionYear', 'TransactionMonth', 'TransactionDay',
       'TransactionHour'],
      dtype='object')

In [18]:
latest_transaction = dff.groupby('CustomerId')['TransactionStartTime_Uganda'].max()
latest_transaction

CustomerId
CustomerId_1      2018-11-21 19:49:14+03:00
CustomerId_10     2018-11-21 19:49:09+03:00
CustomerId_1001   2018-11-16 11:20:39+03:00
CustomerId_1002   2019-01-18 13:05:00+03:00
CustomerId_1003   2019-02-01 18:04:51+03:00
                             ...           
CustomerId_992    2019-02-08 13:27:42+03:00
CustomerId_993    2019-01-18 18:56:30+03:00
CustomerId_994    2019-02-12 14:17:08+03:00
CustomerId_996    2018-12-07 18:24:31+03:00
CustomerId_998    2019-02-13 10:47:23+03:00
Name: TransactionStartTime_Uganda, Length: 3742, dtype: datetime64[ns, Africa/Kampala]

## Model preparing 

In [6]:
categorical_feature = [    'ProviderId', 'ProductId',
                        'ProductCategory', 'ChannelId',  'PricingStrategy']

num_cat = [ 'Amount', 'TransactionYear', 'TransactionMonth',
       'TransactionDay', 'TransactionHour']


In [7]:
all_feature = categorical_feature + num_cat
df_feat = dff[all_feature]
print(df_feat.shape)


(95662, 10)


In [8]:
df_feat.columns

Index(['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId',
       'PricingStrategy', 'Amount', 'TransactionYear', 'TransactionMonth',
       'TransactionDay', 'TransactionHour'],
      dtype='object')

In [133]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProviderId        95662 non-null  object 
 1   ProductId         95662 non-null  object 
 2   ProductCategory   95662 non-null  object 
 3   ChannelId         95662 non-null  object 
 4   PricingStrategy   95662 non-null  int64  
 5   Amount            95662 non-null  float64
 6   TransactionYear   95662 non-null  int32  
 7   TransactionMonth  95662 non-null  int32  
 8   TransactionDay    95662 non-null  int32  
 9   TransactionHour   95662 non-null  int32  
dtypes: float64(1), int32(4), int64(1), object(4)
memory usage: 5.8+ MB


In [16]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
import joblib

In [14]:

def encoder(method, dataframe, columns_featured):
    if method == 'labelEncoder':      
        df_lbl = dataframe.copy()
        for col in columns_featured:
            label = LabelEncoder()
            label.fit(list(dataframe[col].values))
            df_lbl[col] = label.transform(df_lbl[col].values)
        return df_lbl, label 
    
    elif method == 'oneHotEncoder':
        df_lbl = dataframe.copy()
        encoder = OneHotEncoder(sparse_output=False, drop='first')

         # Fit the encoder on the specified columns (ensure 2D input)
        encoder.fit(df_lbl[columns_featured])  # Fit on the entire DataFrame subset

        # Transform the DataFrame
        encoded_array = encoder.transform(df_lbl[columns_featured])

        # Create a DataFrame with the encoded features
        encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(columns_featured))

        # Drop the original columns and concatenate the new encoded columns
        df_lbl = df_lbl.drop(columns=columns_featured).reset_index(drop=True)
        df_lbl = pd.concat([df_lbl, encoded_df], axis=1)

        # Save the fitted encoder
        joblib.dump(encoder, 'one_hot_encoder.pkl')
        
        return df_lbl, encoder

In [20]:
df_label, enoderone  = encoder('oneHotEncoder', df_feat, categorical_feature)

In [21]:
df_label.shape

(95662, 46)

In [22]:
df_scaled = dp.scaler('standardScaler', df_label, num_cat)

In [23]:
df_scaled.shape

(95662, 46)

In [24]:
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [30]:
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Split the data
X = df_scaled.copy()
y = dff['FraudResult']
X_train, X_test, y_train, y_test = split_data(X, y)

In [31]:
# Before fiting the data into the model, lets find the best number of estimators for both models
def mae_score(estimator):
    model1 = RandomForestRegressor(n_estimators= estimator, random_state=0)
    val_score = -1 * cross_val_score(model1, X, y, cv=2, scoring='neg_mean_absolute_error')

    return val_score.mean()

In [32]:
# Create and fit the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [33]:
y_predl = linear_model.predict(X_test)

In [34]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [37]:
 def evaluate_model(model, X_test, y_test):
    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mae, mse, r2, y_pred

models = ['Linear Regression', 'Decision Tree']
mae_scores, mse_scores, r2_scores, y_pred = [], [], [], []
# mae, mse, r2, _ = evaluate_model(dt_model, X_test, y_test)
for model in [linear_model, dt_model]:
    mae, mse, r2, _ = evaluate_model(model, X_test, y_test)
    mae_scores.append(mae)
    mse_scores.append(mse)
    r2_scores.append(r2)
    y_pred.append(_)


In [113]:
print(len(_))
frequency = pd.Series(_).value_counts()
frequency
type(y_test)
# Create a DataFrame
df_test = pd.DataFrame({
    'Regression': y_pred[0],
    'Random Forest': y_pred[1],
    'Actual': y_test})
df_test[df_test['Actual'] == 1]
df_test.head()
# mae_scores
# y_pred

19133


,Regression,Random Forest,Actual
89320,0.001246,0.0,0
60454,-0.000119,0.0,0
59880,-0.000442,0.0,0
43893,0.000033,0.0,0
71812,0.004142,0.0,0


In [107]:
df_test['Actual'].value_counts()

Actual
0    19097
1       36
Name: count, dtype: int64

In [105]:
df_test.describe()

,Regression,Random Forest,Actual
count,19133.000000,19133.000000,19133.000000
mean,0.001881,0.002021,0.001882
std,0.025931,0.044781,0.043337
min,-0.196689,0.000000,0.000000
25%,-0.000350,0.000000,0.000000
50%,0.000138,0.000000,0.000000
75%,0.000755,0.000000,0.000000
max,2.007659,1.000000,1.000000


In [39]:
 print('mae score for Linear Regression and decision tree: ', mae_scores)
 print('mse score for Linear Regression and decision tree: ', mse_scores) # 900, 180  mae
 print('accuracy for Linear Regression and decision tree: ', r2_scores)

mae score for Linear Regression and decision tree:  [0.0033738578155111843, 0.0002439066882698305]
mse score for Linear Regression and decision tree:  [0.0013507572081077868, 0.00023229208406650524]
accuracy for Linear Regression and decision tree:  [0.28075676138639905, 0.8763104798956853]


In [138]:
df_scaled.head(1)

,Amount,TransactionYear,TransactionMonth,TransactionDay,TransactionHour,ohe_ProviderId_2,ohe_ProviderId_3,ohe_ProviderId_4,ohe_ProviderId_5,ohe_ProviderId_6,ohe_ProductId_10,ohe_ProductId_11,ohe_ProductId_12,ohe_ProductId_13,ohe_ProductId_14,ohe_ProductId_15,ohe_ProductId_16,ohe_ProductId_19,ohe_ProductId_2,ohe_ProductId_20,ohe_ProductId_21,ohe_ProductId_22,ohe_ProductId_23,ohe_ProductId_24,ohe_ProductId_27,ohe_ProductId_3,ohe_ProductId_4,ohe_ProductId_5,ohe_ProductId_6,ohe_ProductId_7,ohe_ProductId_8,ohe_ProductId_9,ohe_data_bundles,ohe_financial_services,ohe_movies,ohe_other,ohe_ticket,ohe_transport,ohe_tv,ohe_utility_bill,ohe_ChannelId_2,ohe_ChannelId_3,ohe_ChannelId_5,ohe_1,ohe_2,ohe_4
0,-0.046371,-0.995432,0.849771,-0.097074,-1.902457,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [46]:
df_feat.columns

Index(['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId',
       'PricingStrategy', 'Amount', 'TransactionYear', 'TransactionMonth',
       'TransactionDay', 'TransactionHour'],
      dtype='object')

In [139]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProviderId        95662 non-null  object 
 1   ProductId         95662 non-null  object 
 2   ProductCategory   95662 non-null  object 
 3   ChannelId         95662 non-null  object 
 4   PricingStrategy   95662 non-null  int64  
 5   Amount            95662 non-null  float64
 6   TransactionYear   95662 non-null  int32  
 7   TransactionMonth  95662 non-null  int32  
 8   TransactionDay    95662 non-null  int32  
 9   TransactionHour   95662 non-null  int32  
dtypes: float64(1), int32(4), int64(1), object(4)
memory usage: 5.8+ MB


In [43]:
di = {
  "ProviderId": "ProviderId_6",
  "ProductId": "ProductId_10",
  "ProductCategory": "airtime",
  "ChannelId": "ChannelId_3",
  "PricingStrategy": 2,
  "Amount": 1000,
  "TransactionYear": 2019,
  "TransactionMonth": 1,
  "TransactionDay": 10,
  "TransactionHour": 3
}

In [42]:
encoder = joblib.load('one_hot_encoder.pkl')
scaler = joblib.load('standard_scalar.pkl')

In [44]:
input_data = pd.DataFrame([di])

In [48]:
 encoded_data = encoder.transform(input_data[['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId','PricingStrategy']])

In [50]:
# Scale the numerical features
scaled_data = scaler.transform(input_data[['Amount', 'TransactionYear', 'TransactionMonth',
       'TransactionDay', 'TransactionHour']])

In [51]:
# Combine processed features
processed_data = np.hstack((encoded_data, scaled_data))

# Make predictions
predictions = model.predict(processed_data)

{"prediction": predictions[0]}

{'prediction': 0.0}

In [143]:
input_data = pd.DataFrame([di])
# Encode categorical variables
categorical_feature1 = ['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId']
num_cat1 = [ 'Amount', 'TransactionYear', 'TransactionMonth',
'TransactionDay', 'TransactionHour']
df_label1 = dp.encoder('oneHotEncoder', input_data, categorical_feature1)

df_scaled1 = dp.scaler('standardScaler', df_label1, num_cat1)

In [40]:
import pickle
from datetime import datetime

# Serialize the model with time stamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"model_{timestamp}.pkl"

# Save the model to a pickle file
with open(model_filename, "wb") as file:
    pickle.dump(dt_model, file)
print(f"Model saved as {model_filename}")

Model saved as model_20241008_214241.pkl
